<a href="https://colab.research.google.com/github/meskeremg/FinalCapstone/blob/main/Step_4_2_NLP_Book_Recommendation_Gensim_and_Spacy_Meskerem_Goshime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 4-2: NLP Book Recommendation System
# Model 2 - Gensim and Spacy

Amazon Books Reviews Data data source: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=books_data.csv This is a rich dataset for Natural Language Processing containing 3,000,000 text reviews from users as well as text descriptions and categories for 212,403 books. Therefore it is ideal for text analysis.

# Importing libraries and reading the data

In [ ]:
!python3 -m spacy download en_core_web_lg

2022-11-30 00:29:56.823552: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 14 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import numpy as np
import os
import gensim
import spacy
import en_core_web_lg
nlp = spacy.load('en_core_web_lg')


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fileDownloaded = drive.CreateFile({'id':'1dnURk-tdodpKuv-3Ic92ELyNoQLs9tLA'})
fileDownloaded.GetContentFile('books_after_preprocessing.csv')

In [ ]:
books = pd.read_csv('books_after_preprocessing.csv')

In [ ]:
books.head(3)

,index,Title,review/score_Avg,review/score_Count,authors,publishedDate,description_categories
0,74190,and poetry is born russian classical poetry,4.0,1.0,['Aleksandr Sergeevich Pushkin'],1984.0,russian poetry selection russian poem russian ...
1,80644,and still king,4.0,1.0,['Keith Checkley'],2012.0,business economics nothing provides clearer pi...
2,31352,dancers in mourning,4.5,8.0,['Margery Allingham'],2015.0,fiction murder take center stage songanddance ...


# Taking a subset of the data by selecting the books which received more than 10 reviews

I am taking a subset of the book data to preform Sentence Embeddings and Cosine Similarity. The full dataset would be too large to process.

In [ ]:
books_sm_10 = books[books['review/score_Count'] > 10]
books_sm_10.head(3)

,index,Title,review/score_Avg,review/score_Count,authors,publishedDate,description_categories
30,95768,1 is one,4.866667,30.0,['Tasha Tudor'],2015.0,juvenile nonfiction rhyming verse present numb...
34,76202,1 ragged ridge road,4.277778,18.0,"['Leonard Foglia', 'David Richards']",1998.0,fiction estranged husband carol robbins young ...
36,110134,10 button book,3.142857,28.0,['William Accorsi'],1999.0,juvenile nonfiction verse introduce number one...


In [ ]:
books_sm_10 = books_sm_10.reset_index(drop=True)
books_sm_10 = books_sm_10.drop(columns='index')
books_sm_10.head(3)


,Title,review/score_Avg,review/score_Count,authors,publishedDate,description_categories
0,1 is one,4.866667,30.0,['Tasha Tudor'],2015.0,juvenile nonfiction rhyming verse present numb...
1,1 ragged ridge road,4.277778,18.0,"['Leonard Foglia', 'David Richards']",1998.0,fiction estranged husband carol robbins young ...
2,10 button book,3.142857,28.0,['William Accorsi'],1999.0,juvenile nonfiction verse introduce number one...


In [ ]:
books_sm_10.shape

(29560, 6)

# Creating vectors from the description_categories column

In [ ]:
# Source: https://www.youtube.com/watch?v=Il04RjS-9-8&t=262s

# Creating a column for vectors
books_sm_10['vectors'] = ''

In [ ]:
# Create vectors from the description_categories column
# This took about 9 minutes to run.

for i in range(len(books_sm_10['description_categories'])):
    books_sm_10['vectors'][i] = nlp(books_sm_10['description_categories'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
books_sm_10['vectors'].head()

0    (juvenile, nonfiction, rhyming, verse, present...
1    (fiction, estranged, husband, carol, robbins, ...
2    (juvenile, nonfiction, verse, introduce, numbe...
3    (family, relationship, repeat, everything, eve...
4    (reference, jumpstart, reading, skill, speed, ...
Name: vectors, dtype: object

# Computing similarity scores between the book in the 0 index place and all the books

In [ ]:
# Similarity between books in the 0 and 1 index place


books_sm_10['vectors'][0].similarity(books_sm_10['vectors'][1])

0.594082225178606

In [ ]:
# computing similarity scores between the book in the 0 index place and all of the books, including itself
# I expected this cell to take a while to run. However, it run in few seconds. 
# However, please note that we are computing similarity for only one book at a time, not the whole dataset at once as
# in the case of SBERT Sentence Transformers.

sims = []
index = []
for i in range(len(books_sm_10['vectors'])):
  sim = books_sm_10['vectors'][i].similarity(books_sm_10['vectors'][0])
  sims.append(sim)
  index.append(i)

sims_df = pd.DataFrame(sims, index=index, columns =['sims'])

In [ ]:
# The top 5 highest similarity scores. 
# I used index [1:6] to exclude the similarity score of the book to itself, which would be a similarity score of 1.
top5 = sims_df.sort_values(by='sims', ascending=False)
top5 = top5[1:6]
top5 = top5.reset_index()
top5

,index,sims
0,28380,0.890873
1,28730,0.881430
2,22012,0.879250
3,6160,0.879250
4,17511,0.875732


In [ ]:
# Taking the top 5 index values
index = top5['index'].values
index

array([28380, 28730, 22012,  6160, 17511])

In [ ]:
print('Chosen book: ')
print(books_sm_10['Title'][0])
print('Recommended books: ')
for i in index:
  print(books_sm_10['Title'][i])

Chosen book: 
1 is one
Recommended books: 
way out in the desert
when we were very young winniethepooh winniethepooh series
the dictionary of the khazars
dictionary of the khazars a lexicon novel in 100000 words english and serbocroatian edition
richard scarrys best mother goose ever


# Creating a Function that searches the top 5 similar books

In [ ]:
def find_similar(Title):

  chosen_index = books_sm_10[books_sm_10['Title'] == Title].index[0]
  #chosen_index

  sims = []
  index = []

  for i in range(len(books_sm_10['vectors'])):
    sim = books_sm_10['vectors'][i].similarity(books_sm_10['vectors'][chosen_index])
    sims.append(sim)
    index.append(i)

  sims_df = pd.DataFrame(sims, index=index, columns =['sims'])
  
  top5 = sims_df.sort_values(by='sims', ascending=False)
  top5 = top5[1:6]
  top5 = top5.reset_index()
  #top5

  index = top5['index'].values

  print('Chosen book: ')
  print(books_sm_10['Title'][chosen_index])
  print('Recommended books: ')
  for i in index:
    print(books_sm_10['Title'][i])

# Example Recommendations with the same books I used to test Model 1 and Model 3

In [ ]:
# Chosen title
chosen_title = '1 is one'
find_similar(chosen_title)

Chosen book: 
1 is one
Recommended books: 
way out in the desert
when we were very young winniethepooh winniethepooh series
the dictionary of the khazars
dictionary of the khazars a lexicon novel in 100000 words english and serbocroatian edition
richard scarrys best mother goose ever


One of the recommendations, **The dictionary of the khazars** is a very different book from the chosen book. It is a 339 page novel for adults about imiaginary people living somewhere in Europe. Unfortunately, this title appears twice in the recommendation with variant titles. The rest of the recommendations are rhyming early childhood picture books like the chosen title. I am not satisfied with this recommendation.

In [ ]:
chosen_title = 'spanish stepbystep'
find_similar(chosen_title)

Chosen book: 
spanish stepbystep
Recommended books: 
teach yourself turkish complete course turkish edition
teach yourself korean complete course korean edition
teach yourself finnish
cliffsquickreview writing grammar usage and style
teach yourself finnish complete course teach yourself language complete courses swedish edition


All of the recommended books are language learning books. However, I would hope to get at least a few for the Spanish language specifically.
I am not happy with this recommendation either.

In [ ]:
chosen_title = 'to kill a mockingbird'
find_similar(chosen_title)

Chosen book: 
to kill a mockingbird
Recommended books: 
the painted bird
painted bird
monkey beach a novel
dead names  the dark history of the necronomicon
lost mans river panther s


The recommended books are similar to the chosen title, **To kill a mockingbird** in that they have some element of crime and mystery. Not such great recommendations though, in my opinion. One, the painted bird is about Native Americans. However, the others don't have the element of multi ethnicity as you would expect from the chosen title. However, I notice that the description text for To kill a mockingbird is not that great. It mostly talks about how popular the book is than describing the story.

In [ ]:
books_sm_10[books_sm_10['Title'] == 'to kill a mockingbird']['description_categories'].values

array(['fiction voted america bestloved novel pbs great american read harper lee pulitzer prizewinning masterwork honor injustice deep southand heroism one man face blind violent hatred one cherished story time kill mockingbird translated forty language sold forty million copy worldwide served basis enormously popular motion picture voted one best novel twentieth century librarian across country gripping heartwrenching wholly remarkable tale comingofage south poisoned virulent prejudice view world great beauty savage inequity eye young girl fathera crusading local lawyerrisks everything defend black man unjustly accused terrible crime'],
      dtype=object)

# How about accuracy matrix

In the absence of labled data, I was not able to quantify the accuracy of the model. However, I have assessed the recommendations for several books manually. This model seem to make recommendation or somewhat similar books. However, in my opinion, it could be better. I have found one of the recommendation for an early childhood picture book to be an adult novel.

Here are my assessments on 3 example recommendations.

**1 is One** - My assessment is **not good**. One of the recommendations, **The Dictionary of the Khazars** is a very different book from the chosen book. It is a 339 page novel. But the rest of the recommendations are rhyming early childhood picture books like the chosen book.


**Spanish Step by Step** - Again my assessment is **not good**. All of the recommended books are language learning books, but none are for the Spanish language.

**To Kill a Mockingbird** - **Good** recommendations. The recommended books are similar to the chosen title in that they have some element of crime and mystery and two of the books have some multi-etinic elements like the chosen title. However, they mostly lack the slavery theme.

Please see the recommendation lists for these books in the above codes.

